# Product Card Generator

This notebook creates a web application for generating product cards using the Gemini API. The interface features:
- A form on the left to collect product details
- Generated code and preview on the right

## Import Required Libraries

In [ ]:
# Import necessary libraries
from flask import Flask, render_template, request, jsonify
import google.generativeai as genai
import os
import json
from dotenv import load_dotenv
import markdown
import html

## Set Up Gemini API Integration

In [ ]:
# Load environment variables and configure the Gemini API
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

# Initialize the Gemini Pro model
model = genai.GenerativeModel('gemini-1.5-pro')

## Card Generation Function

In [ ]:
def generate_product_card(product_data):
    """
    Generate a React component for a product card using Gemini AI.
    
    Args:
        product_data: Dictionary containing product name, description, image URL, etc.
        
    Returns:
        String containing the React component code
    """
    prompt = f"""
    Create a React component for a product card with the following details:
    
    Product Name: {product_data['name']}
    Product Description: {product_data['description']}
    Color Theme: {product_data['colorTheme']}
    
    Additional Design Instructions: {product_data['designPrompt']}
    
    {'The component should include an image with the following URL: ' + product_data['imageUrl'] if product_data['imageUrl'] else 'The component should include an image placeholder.'}
    
    Please provide a React component that:
    1. Uses Tailwind CSS for styling
    2. Is responsive and looks good on all devices
    3. Includes appropriate animations or hover effects
    4. Follows modern design principles
    5. Includes a "Buy Now" or "Add to Cart" button
    
    Return only the component code without any explanations.
    """
    
    response = model.generate_content(prompt)
    code = response.text
    
    # Extract code if wrapped in markdown code blocks
    if code.startswith("```") and code.endswith("```"):
        code = code[code.find("\n")+1:code.rfind("```")]
    
    # Remove language identifier if present
    if code.startswith("jsx") or code.startswith("tsx") or code.startswith("javascript"):
        code = code[code.find("\n")+1:]
    
    return code

## React Component Layout

In [ ]:
// BrandSettings.tsx component structure
import { useState } from 'react';
import { Tabs, TabsContent, TabsList, TabsTrigger } from "./ui/tabs";
import { Button } from "./ui/button";
import { generateProductCard } from '../lib/ai';
import { Editor } from './Editor';

export default function BrandSettings() {
  const [formData, setFormData] = useState({
    name: '',
    description: '',
    imageUrl: '',
    colorTheme: 'blue',
    designPrompt: ''
  });
  const [generatedCode, setGeneratedCode] = useState('');
  const [isGenerating, setIsGenerating] = useState(false);
  
  const handleSubmit = async (e) => {
    e.preventDefault();
    setIsGenerating(true);
    
    try {
      const code = await generateProductCard(formData);
      setGeneratedCode(code);
    } catch (error) {
      console.error('Error generating card:', error);
    } finally {
      setIsGenerating(false);
    }
  };
  
  return (
    <div className="flex h-full w-full gap-4 p-4">
      {/* Form card (left) */}
      <div className="w-1/3 neobrutalist-card p-4 h-full overflow-y-auto">
        <h2 className="text-xl font-bold mb-6">Product Card Generator</h2>
        <form onSubmit={handleSubmit}>
          {/* Form fields */}
        </form>
      </div>
      
      {/* Code and Preview card (right) */}
      <div className="w-2/3 neobrutalist-card h-full overflow-hidden">
        <Tabs defaultValue="code" className="w-full h-full">
          <TabsList className="w-full">
            <TabsTrigger value="code" className="flex-1">Code</TabsTrigger>
            <TabsTrigger value="preview" className="flex-1">Preview</TabsTrigger>
          </TabsList>
          <TabsContent value="code" className="h-[calc(100%-3rem)] overflow-hidden">
            <Editor value={generatedCode} language="javascript" />
          </TabsContent>
          <TabsContent value="preview" className="h-[calc(100%-3rem)] overflow-auto p-4">
            {generatedCode ? (
              <div className="preview-container">
                {/* Preview container for the generated component */}
              </div>
            ) : (
              <div className="flex items-center justify-center h-full">
                <p className="text-muted-foreground">Generate a product card to see a preview</p>
              </div>
            )}
          </TabsContent>
        </Tabs>
      </div>
    </div>
  );
}

## Implementation Steps

1. Create a BrandSettings.tsx component in the components directory
2. Create the generateProductCard function in lib/ai.ts
3. Update App.tsx to render BrandSettings when toggled
4. Create an Editor.tsx component for code display
5. Modify the sidebar to include a Product Card Generator option